In [1]:
import tqdm
import time
import warnings
import multiprocessing
import cvxpy as cp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import partial

In [ ]:
time.sleep(3600)

In [ ]:
def check_feasibility(L, a, b):
    x = cp.Variable(len(a))
    constraints = [x <= 2*(b - L)/a, x >= 0, cp.sum(x) == L]
    problem = cp.Problem(cp.Maximize(0), constraints)
    problem.solve()    
    return problem.status in {"optimal", "feasible"}


def solve_fixed_load(L, a, b, metric):

    try: 
        warnings.simplefilter("error")
        if not check_feasibility(L, a, b):
            raise ValueError("infeasible")

        x = cp.Variable(len(a))

        if (metric == "sw") or (metric == 0):
            objective = cp.Maximize(cp.sum(-0.5*cp.multiply(a, x**2) + cp.multiply(b - L, x)))
        elif (metric == "pf") or (metric == 1):
            objective = cp.Maximize(cp.sum(cp.log(x) + cp.log(cp.multiply(-0.5*a, x) + b - L)))
        elif (metric == "mm") or (metric == float("inf")):
            objective = cp.Maximize(cp.min(-0.5*cp.multiply(a, x**2) + cp.multiply(b - L, x)))
        else:
            y = -0.5*cp.multiply(a, x**2) + cp.multiply(b - L, x)
            objective = cp.Maximize(cp.sum(cp.power(y, 1 - metric))/(1 - metric))

        constraints = [x <= 2*(b - L)/a, x >= 0, cp.sum(x) == L]
        problem = cp.Problem(objective, constraints)
        problem.solve(solver="ECOS", abstol=1e-6)

        user_surplus = -0.5*a*x.value**2 + (b - L)*x.value
        surplus = np.sum(user_surplus)

        return problem.value, x.value, user_surplus, surplus, "success"
    
    except (Warning, cp.SolverError, TypeError, ValueError) as e:
        return np.nan, np.full(len(a), np.nan), np.full(len(a), np.nan), np.nan, str(e)


def solve(a, b, metric):
    pool = multiprocessing.Pool(min(multiprocessing.cpu_count(), 100))
    results = pool.map(partial(solve_fixed_load, a=a, b=b, metric=metric), np.arange(0.1, 100, 0.1))
    results = [x for x in results if not np.isnan(x[0])]
    pool.close()
    return sorted(results, key=lambda x: -x[0])[0]


def solve_wrapper(params):
    
    seed = params["seed"]
    n_users = params["n_users"]
    metric = params["metric"]
    xbar = params["xbar"]
    
    np.random.seed(seed)
    
    a = 1*np.random.rand(n_users) + 1
    b = xbar * a
    
    val, sol, user_surplus, surplus, message = solve(a, b, metric)
        
    return dict(
        n_users=n_users, seed=seed, metric=metric, message=message,
        a=a, b=b, val=val, sol=sol, user_surplus=user_surplus,
    )

In [ ]:
%%time

params_list = [
    dict(n_users=n_users, seed=seed, metric=metric, xbar=2)
    for n_users in [100] for seed in range(1000)
    for metric in ["sw", "pf", "mm"]
]

results = [solve_wrapper(params) for params in tqdm.tqdm(params_list)]
df = pd.DataFrame(results)

In [ ]:
df = pd.DataFrame(results)
df["a_rank"] = df.a.apply(lambda x: list(pd.Series(x).rank()))
df_sw = df.loc[df.metric=="sw"]
df_pf = df.loc[df.metric=="pf"]

a = np.array(df_sw.a.tolist())
rel_a = (a / a.max(1, keepdims=True)).flatten()

sw_sol = np.array(df_sw.sol.tolist()).flatten()
sw_surplus = np.array(df_sw.user_surplus.tolist()).flatten()

pf_sol = np.array(df_pf.sol.tolist()).flatten()
pf_surplus = np.array(df_pf.user_surplus.tolist()).flatten()

In [ ]:
plt.figure(figsize=(7, 6))

plt.subplot(2, 2, 1)
plt.plot(rel_a, sw_sol, ".", alpha=0.5)
plt.xlabel("a_i / max(a)")
plt.ylabel("SW Allocation")
plt.grid()

plt.subplot(2, 2, 2)
plt.plot(rel_a, sw_surplus, ".", alpha=0.5)
plt.xlabel("a_i / max(a)")
plt.ylabel("SW Surplus")
plt.grid()


plt.subplot(2, 2, 3)
plt.plot(rel_a, pf_sol, ".", alpha=0.5)
plt.xlabel("a_i / max(a)")
plt.ylabel("PF Allocation")
plt.grid()

plt.subplot(2, 2, 4)
plt.plot(rel_a, pf_surplus, ".", alpha=0.5)
plt.xlabel("a_i / max(a)")
plt.ylabel("PF Surplus")
plt.grid()

plt.tight_layout()
plt.show()

In [ ]:
sw_sol = np.vstack(df.loc[df.metric=="sw", "sol"].tolist()).flatten()
pf_sol = np.vstack(df.loc[df.metric=="pf", "sol"].tolist()).flatten()

data_min = min(min(sw_sol), min(pf_sol))
data_max = max(max(sw_sol), max(pf_sol))
bins = np.linspace(data_min, data_max, 30)

plt.figure(figsize=(4, 3))
plt.hist(sw_sol, bins=bins, color="C0", alpha=0.5, label="SW")
plt.hist(pf_sol, bins=bins, color="C1", alpha=0.5, label="PF")
plt.xlabel("Allocation")
plt.ylabel("Count")
plt.legend()
plt.show()

In [ ]:
sw_surplus = np.vstack(df.loc[df.metric=="sw", "user_surplus"].tolist()).flatten()
pf_surplus = np.vstack(df.loc[df.metric=="pf", "user_surplus"].tolist()).flatten()

data_min = min(min(sw_sol), min(pf_sol))
data_max = max(max(sw_sol), max(pf_sol))
bins = np.linspace(data_min, data_max, 30)

plt.figure(figsize=(4, 3))
plt.hist(sw_surplus, bins=bins, color="C0", alpha=0.5, label="SW")
plt.hist(pf_surplus, bins=bins, color="C1", alpha=0.5, label="PF")
plt.xlabel("Surplus")
plt.ylabel("Count")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(4, 3))
plt.hist(pf_sol - sw_sol, bins=30, color="C0", alpha=0.5)
plt.xlabel("Allocation Gain")
plt.ylabel("Count")
plt.show()

plt.figure(figsize=(4, 3))
plt.hist(pf_surplus - sw_surplus, bins=30, color="C0", alpha=0.5)
plt.xlabel("Surplus Gain")
plt.ylabel("Count")
plt.show()

In [ ]:
np.mean(pf_sol - sw_sol), np.mean(pf_sol > sw_sol)

In [ ]:
np.mean(pf_surplus - sw_surplus), np.mean(pf_surplus > sw_surplus)